In [2]:
!pip install opendatasets

In [4]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/karakaggle/kaggle-cat-vs-dog-dataset')

Dataset URL: https://www.kaggle.com/datasets/karakaggle/kaggle-cat-vs-dog-dataset


100%|██████████| 787M/787M [00:06<00:00, 118MB/s] 


In [6]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten

In [7]:
train_ds=keras.utils.image_dataset_from_directory(
    directory='/content/kaggle-cat-vs-dog-dataset/kagglecatsanddogs_3367a/PetImages',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256))

Found 24959 files belonging to 2 classes.


In [8]:
def process(image,label):
  image=tf.cast(image/255. ,tf.float32)
  return image,label
train_ds=train_ds.map(process)

In [9]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,847,297 (56.64 MB)

 Trainable params: 14,847,297 (56.64 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [13]:
import os

data_dir = '/content/kaggle-cat-vs-dog-dataset/kagglecatsanddogs_3367a/PetImages'

for subdir in ['Cat', 'Dog']:
    subdir_path = os.path.join(data_dir, subdir)
    for filename in os.listdir(subdir_path):
        filepath = os.path.join(subdir_path, filename)
        try:
            img = tf.io.read_file(filepath)
            img = tf.image.decode_image(img, channels=3)
        except Exception as e:
            print(f"Deleting corrupted file: {filepath} due to error: {e}")
            os.remove(filepath)

Deleting corrupted file: /content/kaggle-cat-vs-dog-dataset/kagglecatsanddogs_3367a/PetImages/Cat/4351.jpg due to error: {{function_node __wrapped__DecodeImage_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input size should match (header_size + row_size * abs_height) but they differ by 2 [Op:DecodeImage] name: 
Deleting corrupted file: /content/kaggle-cat-vs-dog-dataset/kagglecatsanddogs_3367a/PetImages/Cat/Thumbs.db due to error: {{function_node __wrapped__DecodeImage_device_/job:localhost/replica:0/task:0/device:CPU:0}} Unknown image file format. One of JPEG, PNG, GIF, BMP required. [Op:DecodeImage] name: 
Deleting corrupted file: /content/kaggle-cat-vs-dog-dataset/kagglecatsanddogs_3367a/PetImages/Dog/2494.jpg due to error: {{function_node __wrapped__DecodeImage_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input size should match (header_size + row_size * abs_height) but they differ by 2 [Op:DecodeImage] name: 
Deleting corrupted file: /content/kaggle-cat-vs-dog-dat

In [15]:
train_ds = keras.utils.image_dataset_from_directory(
    directory='/content/kaggle-cat-vs-dog-dataset/kagglecatsanddogs_3367a/PetImages',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256, 256))
train_ds = train_ds.map(process)

Found 24953 files belonging to 2 classes.


In [18]:
total_batches = tf.data.experimental.cardinality(train_ds).numpy()
train_batches = int(total_batches * 0.8)
val_batches = total_batches - train_batches

val_ds = train_ds.skip(train_batches)
train_ds = train_ds.take(train_batches)

print(f"Number of training batches: {tf.data.experimental.cardinality(train_ds).numpy()}")
print(f"Number of validation batches: {tf.data.experimental.cardinality(val_ds).numpy()}")

Number of training batches: 624
Number of validation batches: 156


In [20]:
model.fit(train_ds,epochs=10,verbose=1,validation_data=val_ds)

Epoch 1/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 84s 135ms/step - accuracy: 0.9878 - loss: 0.0350 - val_accuracy: 0.9480 - val_loss: 0.1593
Epoch 2/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 76s 122ms/step - accuracy: 0.9871 - loss: 0.0373 - val_accuracy: 0.9376 - val_loss: 0.2292
Epoch 3/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 79s 126ms/step - accuracy: 0.9893 - loss: 0.0352 - val_accuracy: 0.9484 - val_loss: 0.1948
Epoch 4/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 76s 122ms/step - accuracy: 0.9938 - loss: 0.0227 - val_accuracy: 0.9414 - val_loss: 0.2455
Epoch 5/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 76s 122ms/step - accuracy: 0.9901 - loss: 0.0321 - val_accuracy: 0.9478 - val_loss: 0.1895
Epoch 6/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 76s 123ms/step - accuracy: 0.9937 - loss: 0.0239 - val_accuracy: 0.9422 - val_loss: 0.2514
Epoch 7/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 76s 123ms/step - accuracy: 0.9942 - loss: 0.0185 - val_accuracy: 0.9145 - val_loss: 0.3465
Epoch 8/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 82s 131ms/step - accuracy: 0.9939 - loss: 0